# Quick start guide
This notebook serves as an example of how to train a simple model using pytorch and the ready-to-train AI4Arctic challenge dataset. Initially, a dictionary, 'train_options', is set up with relevant options for both the example U-Net Convolutional Neural Network model and the dataloader. Note that the weights of the U-Net will be initialised at random and therefore not deterministic - results will vary for every training run. Two lists (dataset.json and testset.json) include the names of the scenes relevant to training and testing, where the former can be altered if desired. Training data is loaded in parallel using the build-in torch Dataset and Dataloader classes, and works by randomly sampling a scene and performing a random crop to extract a patch. Each batch will then be compiled of X number of these patches with the patch size in the 'train_options'. An obstacle is different grid resolution sizes, which is overcome by upsampling low resolution variables, e.g. AMSR2, ERA5, to match the SAR pixels. A number of batches will be prepared in parallel and stored until use, depending on the number of workers (processes) spawned (this can be changed in 'num_workers' in 'train_options'). The model is trained on a fixed number of steps according to the number of batches in an epoch, defined by the 'epoch_len' parameter, and will run for a total number of epochs depending on the 'epochs' parameter. After each epoch, the model is evaluated. In this example, a random number of scenes are sampled among the training scenes (and removed from the list of training scenes) to act as a validation set used for the evaluation. The model is evaluated with the metrics, and if the current validation attempt is superior to the previous, then the model parameters are stored in the 'best_model' file in the directory.

The models are scored on the three sea ice parameters; Sea Ice Concentration (SIC), Stage of Development (SOD) and the Floe size (FLOE) with the $R²$ metric for the SIC, and the weighted F1 metric for the SOD and FLOE. The 3 scores are combined into a single metric by taking the weighted average with SIC and SOD being weighted with 2 and the FLOE with 1.

Finally, once you are ready to test your model on the test scenes (without reference data), the 'test_upload' notebook will produce model outputs with your model of choice and save the output as a netCDF file, which can be uploaded to the AI4EO.eu website. The model outputs will be evaluated and then you will receive a score. 

This quick start notebook is by no means necessary to utilize, and you are more than welcome to develop your own data pipeline. We do however require that the model output is stored in a netcdf file with xarray.dataarrays titled '{scene_name}_{chart}', i.e. 3 charts per scene / file (see how in 'test_upload'). In addition, you are more than welcome to create your own preprocessing scheme to prepare the raw AI4Arctic challenge dataset. However, we ask that the model output is in 80 m pixel spacing (original is 40 m), and that you follow the class numberings from the lookup tables in 'utils' - at least you will be evaluated in this way. Furthermore, we have included a function to convert the polygon_icechart to SIC, SOD and FLOE, you will have to incorporate it yourself.

The first cell imports the necessary Python packages, initializes the 'train_options' dictionary, the sample U-Net options, loads the dataset list and select validation scenes.

In [2]:
# -- Built-in modules -- #
import gc
import os
import sys

# -- Environmental variables -- #
#os.environ['AI4ARCTIC_DATA'] = ''  # Fill in directory for data location.
#os.environ['AI4ARCTIC_ENV'] = ''  # Fill in directory for environment with Ai4Arctic get-started package. 


In [4]:
print(os.environ['AI4ARCTIC_DATA'])
print(os.environ)

/cache/ai4arctic2022/asid3_challenge_data_prep/
environ({'SHELL': '/bin/bash', 'NVIDIA_VISIBLE_DEVICES': 'all', 'KUBERNETES_SERVICE_PORT_HTTPS': '443', 'MLFLOW_B94AD1BA_1D5E_4364_881B_D9ADC9064226_PORT_80_TCP_PORT': '80', 'MLFLOW_418974FE_C77F_41E8_ABA2_ADBCD426A9CF_PORT_80_TCP': 'tcp://172.20.41.116:80', 'MLFLOW_B70AF4AD_FF6E_46BB_83BC_D4B7651B2A47_PORT_80_TCP_PORT': '80', 'MLFLOW_EE3104D0_2718_4015_938D_F042E4426B06_SERVICE_HOST': '172.20.114.74', 'JUPYTERHUB_ADMIN_ACCESS': '1', 'MLFLOW_088DC101_03F4_4AAB_AD66_39D5377A7AAA_SERVICE_PORT_HTTP': '80', 'MLFLOW_98B7B842_CE8A_4C98_AFE9_06881D29CCF7_SERVICE_PORT': '80', 'MLFLOW_708F7831_7A58_4B62_B09F_32B6B29099CD_PORT_80_TCP': 'tcp://172.20.50.165:80', 'KUBERNETES_SERVICE_PORT': '443', 'NBVIEWER_PORT_8080_TCP': 'tcp://172.20.14.162:8080', 'MLFLOW_AEEB12B0_514D_45C8_9206_6AF134187C47_SERVICE_HOST': '172.20.15.81', 'VOLUME_NFS_PORT_2049_TCP': 'tcp://172.20.203.80:2049', 'MLFLOW_A19682E0_9B3A_4017_BE2B_6537A8D263EE_PORT': 'tcp://172.20.102.17

In [3]:
# -- Third-part modules -- #
import json
import matplotlib.pyplot as plt
import numpy as np
import torch
import xarray as xr
from tqdm.notebook import tqdm  # Progress bar
import mlflow
import mlflow.pytorch

# --Proprietary modules -- #
from functions import chart_cbar, r2_metric, f1_metric, compute_metrics  # Functions to calculate metrics and show the relevant chart colorbar.
from loaders import AI4ArcticChallengeDataset, AI4ArcticChallengeTestDataset, get_variable_options  # Custom dataloaders for regular training and validation.
from unet import UNet  # Convolutional Neural Network model
from utils import CHARTS, SIC_LOOKUP, SOD_LOOKUP, FLOE_LOOKUP, SCENE_VARIABLES, colour_str

train_options = {
    # -- Training options -- #
    'path_to_processed_data': os.environ['AI4ARCTIC_DATA'],  # Replace with data directory path.
    'path_to_env': '',  # Replace with environmment directory path.
    'lr': 0.0001,  # Optimizer learning rate.
    'epochs': 50,  # Number of epochs before training stop.
    'epoch_len': 500,  # Number of batches for each epoch.
    'patch_size': 256,  # Size of patches sampled. Used for both Width and Height.
    'batch_size': 8,  # Number of patches for each batch.
    'loader_upsampling': 'nearest',  # How to upscale low resolution variables to high resolution.
    
    # -- Data prepraration lookups and metrics.
    'train_variables': SCENE_VARIABLES,  # Contains the relevant variables in the scenes.
    'charts': CHARTS,  # Charts to train on.
    'n_classes': {  # number of total classes in the reference charts, including the mask.
        'SIC': SIC_LOOKUP['n_classes'],
        'SOD': SOD_LOOKUP['n_classes'],
        'FLOE': FLOE_LOOKUP['n_classes']
    },
    'pixel_spacing': 80,  # SAR pixel spacing. 80 for the ready-to-train AI4Arctic Challenge dataset.
    'train_fill_value': 0,  # Mask value for SAR training data.
    'class_fill_values': {  # Mask value for class/reference data.
        'SIC': SIC_LOOKUP['mask'],
        'SOD': SOD_LOOKUP['mask'],
        'FLOE': FLOE_LOOKUP['mask'],
    },
    
    # -- Validation options -- #
    'chart_metric': {  # Metric functions for each ice parameter and the associated weight.
        'SIC': {
            'func': r2_metric,
            'weight': 2,
        },
        'SOD': {
            'func': f1_metric,
            'weight': 2,
        },
        'FLOE': {
            'func': f1_metric,
            'weight': 1,
        },
    },
    'num_val_scenes': 10,  # Number of scenes randomly sampled from train_list to use in validation.
    
    # -- GPU/cuda options -- #
    'gpu_id': 0,  # Index of GPU. In case of multiple GPUs.
    'num_workers': 6,  # Number of parallel processes to fetch data.
    'num_workers_val': 1,  # Number of parallel processes during validation.
    
    # -- U-Net Options -- # now 3 lvls as in the paper
    # DIFF:
    # 1. run: 'unet_conv_filters': [16, 32, 32, 3],
    'unet_conv_filters': [16, 32, 32, 32, 32, 32, 32, 32],     # Number of filters in the U-Net.
    'conv_kernel_size': (3, 3),  # Size of convolutional kernels.
    'conv_stride_rate': (1, 1),  # Stride rate of convolutional kernels.
    'conv_dilation_rate': (1, 1),  # Dilation rate of convolutional kernels.
    'conv_padding': (1, 1),  # Number of padded pixels in convolutional layers.
    'conv_padding_style': 'zeros',  # Style of padding.
}
# Get options for variables, amsrenv grid, cropping and upsampling.
get_variable_options = get_variable_options(train_options)
# To be used in test_upload.
%store train_options  

# Load training list.
with open(train_options['path_to_env'] + 'datalists/dataset.json') as file:
    train_options['train_list'] = json.loads(file.read())
# Convert the original scene names to the preprocessed names.
train_options['train_list'] = [file[17:32] + '_' + file[77:80] + '_prep.nc' for file in train_options['train_list']]
# Select a random number of validation scenes with the same seed. Feel free to change the seed.et
np.random.seed(0)
train_options['validate_list'] = np.random.choice(np.array(train_options['train_list']), size=train_options['num_val_scenes'], replace=False)
# Remove the validation scenes from the train list.
train_options['train_list'] = [scene for scene in train_options['train_list'] if scene not in train_options['validate_list']]
print('Options initialised')


Stored 'train_options' (dict)
Options initialised


### CUDA / GPU Setup
This sets up the 'device' variable containing GPU information, and the custom dataset and dataloader.

In [4]:
# Get GPU resources.
if torch.cuda.is_available():
    print(colour_str('GPU available!', 'green'))
    print('Total number of available devices: ', colour_str(torch.cuda.device_count(), 'orange'))
    device = torch.device(f"cuda:{train_options['gpu_id']}")

else:
    print(colour_str('GPU not available.', 'red'))
    device = torch.device('cpu')

# Custom dataset and dataloader.
dataset = AI4ArcticChallengeDataset(files=train_options['train_list'], options=train_options)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=None, shuffle=True, num_workers=train_options['num_workers'], pin_memory=True)
# - Setup of the validation dataset/dataloader. The same is used for model testing in 'test_upload.ipynb'.
dataset_val = AI4ArcticChallengeTestDataset(options=train_options, files=train_options['validate_list'])
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)

print('GPU and data setup complete.')

GPU not available.
GPU and data setup complete.


### Example of Model, optimiser and loss function setup

In [5]:
# Setup U-Net model, adam optimizer, loss function and dataloader.
net = UNet(options=train_options).to(device)
optimizer = torch.optim.Adam(list(net.parameters()), lr=train_options['lr'])
torch.backends.cudnn.benchmark = True  # Selects the kernel with the best performance for the GPU and given input size.

# Loss functions to use for each sea ice parameter.
# The ignore_index argument discounts the masked values, ensuring that the model is not using these pixels to train on.
# It is equivalent to multiplying the loss of the relevant masked pixel with 0.
loss_functions = {chart: torch.nn.CrossEntropyLoss(ignore_index=train_options['class_fill_values'][chart]) \
                                                   for chart in train_options['charts']}
print('Model setup complete')

Model setup complete


## Example of model training and validation loop
A simple model training loop following by a simple validation loop. Validation is carried out on full scenes, i.e. no cropping or stitching. If there is not enough space on the GPU, then try to do it on the cpu. This can be done by using 'net = net.cpu()'.

### Workflow with MLflow

- every model needs to be initiated with start_run() function
- parameters can be logged with log_params() method
- metrics are added with log_metric() with specified name and value
- model can be stored into the model storage as artifact using corresponding flavour
- for using MLflow autologing, model needs to use one of the supported flavours, eg. pytorch-lightning, fastai, tensorflow, xgboost or others - see [full list](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging) 


In [6]:
## setting up the sqlite database for tracking of experiments in MLflow
mlflow.set_tracking_uri('sqlite:///' + os.path.expanduser(os.environ["MLFLOW_BACKEND_STORE_PATH"]))
os.path.expanduser(os.environ["MLFLOW_BACKEND_STORE_PATH"])

## setting the used experiment - if do not exist, new one will be created
e = mlflow.set_experiment("pizza funghi")
e.experiment_id

'1'

In [ ]:
# implementing an ealy stopper
class EarlyStopper:
    """
    Stops the training early iff the validation loss does not increase anymore more than a small delta.
    As the FLOE is weighted only half as much as the other two, the model does only need to get better half as much there.
    """
    def __init__(self, patience=1, min_delta=0.1):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_val_score = -np.inf
        
    def early_stop(self, combined_score, best_combined_score):
        # does not improve enough anymore: < old best + delta
        if combined_score <= (self.max_val_score + self.min_delta):
            self.counter += 1
        else:
            self.max_val_score = best_combined_score # such that there is not a crawling improvement not seen
            self.counter = 0
        return self.counter > self.patience

In [7]:
best_combined_score = 0  # Best weighted model score.
early_stopper = EarlyStopper(patience = 4, min_delta=0.3)

with mlflow.start_run() as run:
    mlflow.log_params(train_options['chart_metric'])
    # -- Training Loop -- #
    for epoch in tqdm(iterable=range(train_options['epochs']), position=0):
        gc.collect()  # Collect garbage to free memory.
        loss_sum = torch.tensor([0.])  # To sum the batch losses during the epoch.
        net.train()  # Set network to evaluation mode.

        # Loops though batches in queue.
        for i, (batch_x, batch_y) in enumerate(tqdm(iterable=dataloader, total=train_options['epoch_len'], colour='red', position=0)):
            torch.cuda.empty_cache()  # Empties the GPU cache freeing up memory.
            loss_batch = 0  # Reset from previous batch.

            # - Transfer to device.
            batch_x = batch_x.to(device, non_blocking=True)

            # - Mixed precision training. (Saving memory)
            with torch.cuda.amp.autocast():
                # - Forward pass. 
                output = net(batch_x)

                # - Calculate loss.
                for chart in train_options['charts']:
                    loss_batch += loss_functions[chart](input=output[chart], target=batch_y[chart].to(device))
                    mlflow.log_metric(key="chart_loss", value=loss_batch)
                    
            # - Reset gradients from previous pass.
            optimizer.zero_grad()

            # - Backward pass.
            loss_batch.backward()

            # - Optimizer step
            optimizer.step()

            # - Add batch loss.
            loss_sum += loss_batch.detach().item()

            # - Average loss for displaying
            loss_epoch = torch.true_divide(loss_sum, i + 1).detach().item()
            print('\rMean training loss: ' + f'{loss_epoch:.3f}', end='\r')
            mlflow.log_metric(key="mean_loss", value=loss_epoch)
            del output, batch_x, batch_y # Free memory.
        del loss_sum

        # -- Validation Loop -- #
        loss_batch = loss_batch.detach().item()  # For printing after the validation loop.

        # - Stores the output and the reference pixels to calculate the scores after inference on all the scenes.
        outputs_flat = {chart: np.array([]) for chart in train_options['charts']}
        inf_ys_flat = {chart: np.array([]) for chart in train_options['charts']}

        net.eval()  # Set network to evaluation mode.
        # - Loops though scenes in queue.
        for inf_x, inf_y, masks, name in tqdm(iterable=dataloader_val, total=len(train_options['validate_list']), colour='green', position=0):
            torch.cuda.empty_cache()

            # - Ensures that no gradients are calculated, which otherwise take up a lot of space on the GPU.
            with torch.no_grad(), torch.cuda.amp.autocast():
                inf_x = inf_x.to(device, non_blocking=True)
                output = net(inf_x)

            # - Final output layer, and storing of non masked pixels.
            for chart in train_options['charts']:
                output[chart] = torch.argmax(output[chart], dim=1).squeeze().cpu().numpy()
                outputs_flat[chart] = np.append(outputs_flat[chart], output[chart][~masks[chart]])
                inf_ys_flat[chart] = np.append(inf_ys_flat[chart], inf_y[chart][~masks[chart]].numpy())

            del inf_x, inf_y, masks, output  # Free memory.

        # - Compute the relevant scores.
        combined_score, scores = compute_metrics(true=inf_ys_flat, pred=outputs_flat, charts=train_options['charts'],
                                                 metrics=train_options['chart_metric'])

        print("")
        print(f"Final batch loss: {loss_batch:.3f}")
        print(f"Epoch {epoch} score:")
        for chart in train_options['charts']:
            print(f"{chart} {train_options['chart_metric'][chart]['func'].__name__}: {scores[chart]}%")
        print(f"Combined score: {combined_score}%")
        
        mlflow.log_metric(key="final_batch_loss", value=loss_batch)
        mlflow.log_metric(key="combined_score", value=combined_score)

        # If the scores is better than the previous epoch, then save the model and rename the image to best_validation.
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            torch.save(obj={'model_state_dict': net.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'epoch': epoch},
                            f='best_model')
        elif early_stopper(combined_score, best_combined_score):
            break
            
        del inf_ys_flat, outputs_flat  # Free memory.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

/home/conda/autoice/d163ac953e591afe74a6e27636e7584eb349f707f4365d74b1eda380edb75a78-20221130-125246-700756-50-torch_v1/lib/python3.9/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  0%|          | 0/10 [00:00<?, ?it/s]


Final batch loss: 5.675
Epoch 0 score:
SIC r2_metric: -81.538%
SOD f1_metric: 0.406%
FLOE f1_metric: 23.585%
Combined score: -27.736%


...

  0%|          | 0/10 [00:00<?, ?it/s]


Final batch loss: 1.553
Epoch 40 score:
SIC r2_metric: 80.195%
SOD f1_metric: 87.785%
FLOE f1_metric: 84.592%
Combined score: 84.11%


  0%|          | 0/500 [00:00<?, ?it/s]

/home/conda/autoice/d163ac953e591afe74a6e27636e7584eb349f707f4365d74b1eda380edb75a78-20221130-125246-700756-50-torch_v1/lib/python3.9/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


MlflowException: (sqlite3.OperationalError) database is locked
(Background on this error at: https://sqlalche.me/e/14/e3q8)